Lambda School Data Science

*Unit 4, Sprint 1, Module 3*

---

# Document Classification (Assignment)

This notebook is for you to practice skills during lecture.

Today's guided module project and assignment will be different. You already know how to do classification. You ready know how to extract features from documents. So? That means you're ready to combine and practice those skills in a kaggle competition. We we will open with a five minute sprint explaining the competition, and then give you 25 minutes to work. After those twenty five minutes are up, I will give a 5-minute demo an NLP technique that will help you with document classification (*and **maybe** the competition*).

Today's all about having fun and practicing your skills.

## Sections
* <a href="#p1">Part 1</a>: Text Feature Extraction & Classification Pipelines
* <a href="#p2">Part 2</a>: Latent Semantic Indexing
* <a href="#p3">Part 3</a>: Word Embeddings with Spacy
* <a href="#p4">Part 4</a>: Post Lecture Assignment

# Text Feature Extraction & Classification Pipelines (Learn)
<a id="p1"></a>

## Follow Along 

What you should be doing now:
1. Join the Kaggle Competition
2. Download the data
3. Train a model (try using the pipe method I just demoed)

### Load Competition Data

In [1]:
import pandas as pd

# You may need to change the path
train = pd.read_csv('./data/train.csv')
test = pd.read_csv('./data/test.csv')

In [2]:
train.head()

,id,description,ratingCategory
0,1321,"\nSometimes, when whisky is batched, a few lef...",1
1,3861,\nAn uncommon exclusive bottling of a 6 year o...,0
2,655,\nThis release is a port version of Amrut’s In...,1
3,555,\nThis 41 year old single cask was aged in a s...,1
4,1965,"\nQuite herbal on the nose, with aromas of dri...",1


In [3]:
def clean_data(X):
    X = X.copy()
    X['description'] = X['description'].apply(lambda text: text.replace("\n", ""))
    
    return X

In [4]:
train = clean_data(train)
train.head()

,id,description,ratingCategory
0,1321,"Sometimes, when whisky is batched, a few lefto...",1
1,3861,An uncommon exclusive bottling of a 6 year old...,0
2,655,This release is a port version of Amrut’s Inte...,1
3,555,This 41 year old single cask was aged in a she...,1
4,1965,"Quite herbal on the nose, with aromas of dried...",1


In [5]:
test = clean_data(test)

In [6]:
# Distribution of ratingCategory: 0 (Excellent), 1 (Good), 2 (Poor)
train.ratingCategory.value_counts()

1    2881
0    1141
2      65
Name: ratingCategory, dtype: int64

### Splitting training set to Train and Validation


In [7]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(train['description'], 
                                                    train['ratingCategory'], 
                                                    test_size=0.2, 
                                                    stratify=train['ratingCategory'],
                                                    random_state=42)

print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(3269,) (818,) (3269,) (818,)


### Define Pipeline Components

In [8]:
# Import Statements
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
import re

In [9]:
from sklearn.pipeline import Pipeline
vect = TfidfVectorizer(stop_words='english', ngram_range=(1,2))
clf =  RandomForestClassifier()

pipe = Pipeline([('vect', vect), ('clf', clf)])

### Define Your Search Space
You're looking for both the best hyperparameters of your vectorizer and your classification model. 

In [10]:
X_train.head()

1782    Very fragrant, with notes of fresh ripe plum, ...
3414    The good news: This is one of the best Highlan...
2744    A sweet nose of barley sugar, Refreshers candy...
77      Cask # 1013 from Edradour was filled on March ...
354     This whisky packs a lot of clean, complex, and...
Name: description, dtype: object

In [11]:
parameters = {
    'vect__max_df': (0.7, 1.0),
    'vect__min_df': (2, 5, 10),
    #'clf__n_estimators': (100, 500),
    #'clf__max_depth': (10, 20),
    
}

grid_search = GridSearchCV(pipe, parameters, cv=4, n_jobs=-1, verbose=1)
grid_search.fit(X_train, y_train)

Fitting 4 folds for each of 6 candidates, totalling 24 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  24 out of  24 | elapsed:   22.3s finished


GridSearchCV(cv=4, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('vect',
                                        TfidfVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.float64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=True,
                                                        max_df=1.0,
                                                        max_features=None,
                                                        min_df=1,
                                                        ngram_range=(1, 2),
                                                        no

In [12]:
print(grid_search.best_score_)
print(grid_search.best_params_)

0.7399821339326597
{'vect__max_df': 0.7, 'vect__min_df': 5}


In [13]:
from sklearn.metrics import accuracy_score

# Evaluate on test data
score = grid_search.predict(X_test)
accuracy_score(score, y_test)

0.7420537897310513

### Make a Submission File
*Note:* In a typical Kaggle competition, you are only allowed two submissions a day, so you only submit if you feel you cannot achieve higher test accuracy. For this competition the max daily submissions are capped at **20**. Submit for each demo and for your assignment. 

In [24]:
# Predictions on test sample
pred = grid_search.predict(test['description'])

In [25]:
submission = pd.DataFrame({'id': test['id'], 'ratingCategory':pred})
submission['ratingCategory'] = submission['ratingCategory'].astype('int64')

In [26]:
# Make Sure the Category is an Integer
submission.head()

,id,ratingCategory
0,3461,1
1,2604,1
2,3341,1
3,3764,1
4,2306,1


### First Submission

In [29]:
subNumber = 1

In [30]:
# Saving your Submission File
# Best to Use an Integer for different versions of your model

submission.to_csv(f'./data/submission{subNumber}.csv', index=False)
subNumber += 1

## Challenge

You're trying to achieve a minimum of 70% Accuracy on your model.

## Latent Semantic Indexing (Learn)
<a id="p2"></a>

## Follow Along
1. Join the Kaggle Competition
2. Download the data
3. Train a model & try: 
    - Creating a Text Extraction & Classification Pipeline
    - Tune the pipeline with a `GridSearchCV` or `RandomizedSearchCV`
    - Add some Latent Semantic Indexing (lsi) into your pipeline. *Note:* You can grid search a nested pipeline, but you have to use double underscores ie `lsi__svd__n_components`
4. Make a submission to Kaggle 


### Adding the clf esimators and max depth from above submission

In [15]:
parameters = {
    'vect__max_df': (0.7, 1.0),
    'vect__min_df': (2, 5, 10),
    'clf__n_estimators': (100, 500),
      'clf__max_depth': (10, 20),
    
}

grid_search = GridSearchCV(pipe, parameters, cv=4, n_jobs=-1, verbose=1)
grid_search.fit(X_train, y_train)

Fitting 4 folds for each of 24 candidates, totalling 96 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   28.4s
[Parallel(n_jobs=-1)]: Done  96 out of  96 | elapsed:  1.3min finished


GridSearchCV(cv=4, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('vect',
                                        TfidfVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.float64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=True,
                                                        max_df=1.0,
                                                        max_features=None,
                                                        min_df=1,
                                                        ngram_range=(1, 2),
                                                        no

In [16]:
print(grid_search.best_score_)
print(grid_search.best_params_)

0.7228545007825757
{'clf__max_depth': 20, 'clf__n_estimators': 100, 'vect__max_df': 0.7, 'vect__min_df': 10}


In [17]:
from sklearn.metrics import accuracy_score

# Evaluate on test data
score = grid_search.predict(X_test)
accuracy_score(score, y_test)

0.7249388753056235

### Define Pipeline Components

In [31]:
import scipy.stats as stats
from sklearn.model_selection import RandomizedSearchCV
from sklearn.decomposition import TruncatedSVD

# Using parameters from GridSearch from the previous section
vect = TfidfVectorizer(stop_words='english', 
                       ngram_range=(1,2),
                       min_df=10, 
                       max_df=1.0)

# Adding dimensionality reduction
svd = TruncatedSVD(algorithm='randomized',
                   n_iter=10)

# Using parameters from RandomSearch from the previous section
rfc = RandomForestClassifier(n_estimators=500,
                             max_depth=20,
                             random_state=42)

In [32]:
pipe = Pipeline([
    ('vect', vect),      # TF-IDF Vectorizer
    ('svd', svd),        # Truncated SVD Dimensionality Reduction
    ('rfc', rfc)         # RandomForest Classifier
])

### Define Your Search Space
You're looking for both the best hyperparameters of your vectorizer and your classification model. 

In [33]:
params = {
    'svd__n_components': stats.randint(100, 500),
    'vect__max_df': (0.7, 1.0),
    'vect__min_df': (2, 5, 10),
    'rfc__n_estimators': (100, 500),
    'rfc__max_depth': (10, 20)
}

In [34]:
# Fit
random_search = RandomizedSearchCV(pipe, params, cv=2, n_iter=3, n_jobs=-1, verbose=1)
random_search.fit(X_train, y_train)

Fitting 2 folds for each of 3 candidates, totalling 6 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   6 out of   6 | elapsed:   38.8s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   6 out of   6 | elapsed:   38.8s finished


RandomizedSearchCV(cv=2, error_score=nan,
                   estimator=Pipeline(memory=None,
                                      steps=[('vect',
                                              TfidfVectorizer(analyzer='word',
                                                              binary=False,
                                                              decode_error='strict',
                                                              dtype=<class 'numpy.float64'>,
                                                              encoding='utf-8',
                                                              input='content',
                                                              lowercase=True,
                                                              max_df=1.0,
                                                              max_features=None,
                                                              min_df=10,
                                                       

In [35]:
print(random_search.best_score_)
print(random_search.best_params_)

0.7124513866274391
{'rfc__max_depth': 20, 'rfc__n_estimators': 500, 'svd__n_components': 130, 'vect__max_df': 1.0, 'vect__min_df': 10}


In [37]:
# Evaluate on test data
from sklearn.metrics import accuracy_score
score = random_search.predict(X_test)
accuracy_score(score, y_test)

0.7163814180929096

### Make a Submission File

In [38]:
# Predictions on test sample
pred = grid_search.predict(test['description'])

In [39]:
submission = pd.DataFrame({'id': test['id'], 'ratingCategory':pred})
submission['ratingCategory'] = submission['ratingCategory'].astype('int64')

In [40]:
# checking the submission
submission.head()

,id,ratingCategory
0,3461,1
1,2604,1
2,3341,1
3,3764,1
4,2306,1


### Second submission

In [41]:
# Saving the Submission File
# Best to Use an Integer or Timestamp for different versions of your model

submission.to_csv(f'./data/submission{subNumber}.csv', index=False)
subNumber += 2

In [43]:
df = pd.read_csv('./data/submission2.csv')
df.shape


(1022, 2)

In [44]:
df = pd.read_csv('./data/submission1.csv')
df.shape

(1022, 2)

## Challenge

Continue to apply Latent Semantic Indexing (LSI) to various datasets. 

# Word Embeddings with Spacy (Learn)
<a id="p3"></a>

## Follow Along

In [1]:
# Apply to your Dataset

from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import GradientBoostingClassifier

from scipy.stats import randint

param_dist = {
    
    'max_depth' : randint(3,10),
    'min_samples_leaf': randint(2,15)
}

In [45]:
# Continue Word Embedding Work Here
import spacy
nlp = spacy.load("en_core_web_lg")

In [46]:
def get_word_vectors(docs):
    return [nlp(doc).vector for doc in docs]

In [47]:
X = get_word_vectors(X_train)
len(X) == len(X_train)

True

In [48]:
X_test_vec = get_word_vectors(X_test)

In [49]:
rfc.fit(X, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=20, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=500,
                       n_jobs=None, oob_score=False, random_state=42, verbose=0,
                       warm_start=False)

In [50]:
score = rfc.predict(X_test_vec)
accuracy_score(score, y_test)

0.7432762836185819

### Make a Submission File

In [51]:
# Predictions on test sample
pred = rfc.predict(get_word_vectors(test['description']))

In [52]:
submission = pd.DataFrame({'id': test['id'], 'ratingCategory':pred})
submission['ratingCategory'] = submission['ratingCategory'].astype('int64')

In [53]:
# Make Sure the Category is an Integer
submission.head()

,id,ratingCategory
0,3461,1
1,2604,1
2,3341,1
3,3764,1
4,2306,1


### Third submission

In [54]:
# Save your Submission File
# Best to Use an Integer or Timestamp for different versions of your model

submission.to_csv(f'./data/submission{subNumber}.csv', index=False)
subNumber += 3

## Challenge

What you should be doing now:
1. Join the Kaggle Competition
2. Download the data
3. Train a model & try: 
    - Creating a Text Extraction & Classification Pipeline
    - Tune the pipeline with a `GridSearchCV` or `RandomizedSearchCV`
    - Add some Latent Semantic Indexing (lsi) into your pipeline. *Note:* You can grid search a nested pipeline, but you have to use double underscores ie `lsi__svd__n_components`
    - Try to extract word embeddings with Spacy and use those embeddings as your features for a classification model.
4. Make a submission to Kaggle 

# Post Lecture Assignment
<a id="p4"></a>

Your primary assignment this afternoon is to achieve a minimum of 70% accuracy on the Kaggle competition. Once you have achieved 70% accuracy, please work on the following: 

1. Research "Sentiment Analysis". Provide answers in markdown to the following questions: 
    - What is "Sentiment Analysis"? 
    - Is Document Classification different than "Sentiment Analysis"? Provide evidence for your response
    - How do create labeled sentiment data? Are those labels really sentiment?
    - What are common applications of sentiment analysis?
2. Research our why word embeddings worked better for the lecture notebook than on the whiskey competition.
    - This [text classification documentation](https://developers.google.com/machine-learning/guides/text-classification/step-2-5) from Google might be of interest
    - Neural Networks are becoming more popular for document classification. Why is that the case?